In [9]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [10]:
lemmatizer = WordNetLemmatizer()

In [11]:
messages = pd.read_csv('SMSSpamCollection', sep= '\t', names= ['label', 'message'])

In [12]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [13]:
corpus = []

In [14]:
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = nltk.word_tokenize(review)
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
cv = CountVectorizer(max_features = 5000)

data = cv.fit_transform(corpus).toarray()

In [48]:
label = pd.get_dummies(messages['label'])

In [49]:
label = label.iloc[:,1]

In [50]:
print(label)

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: spam, Length: 5572, dtype: uint8


In [51]:
from sklearn.model_selection import train_test_split 

In [70]:
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size = 0.2, random_state = 0) 

In [71]:
from sklearn.naive_bayes import MultinomialNB

In [72]:
spam_detector = MultinomialNB()

spam_dector.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [73]:
pred = spam_dector.predict(x_test)

In [74]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [75]:
accuracy = accuracy_score(y_test, pred)
cf = classification_report(y_test, pred)
cm = confusion_matrix(y_test, pred)

In [76]:
print(accuracy)
print(cf)
print(cm)

0.9820627802690582
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       955
           1       0.93      0.94      0.94       160

    accuracy                           0.98      1115
   macro avg       0.96      0.97      0.96      1115
weighted avg       0.98      0.98      0.98      1115

[[944  11]
 [  9 151]]


In [77]:
lab = pd.DataFrame(label, columns = ['Labels'])

In [78]:
lab.head()

,Labels


In [79]:
label.value_counts()

0    4825
1     747
Name: spam, dtype: int64

In [80]:
messages['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64